# Option Pricing: Asian options and Temperature Derivatives

### Job Marcelis, Ernani Hazbolatow, Koen Verlaan
#### May 2025, University of Amsterdam

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from src.Euler_vs_Milstein import *
from src.control_variate_MC import *

#### Asian options under the Heston model

In [ ]:
# Model parameters
S0 = 100
V0 = 0.04
K = 105
T = 1
dt = 0.001
r = 0.05
rho = -0.7
kappa = 2
theta = 0.04
xi = 0.25
M = 10000
seed = 69
sigma = 0.2

First, we compare the Euler and Milstein discretization schemes:

In [ ]:
dts = np.logspace(-5, -1, 15)
eul_means = np.zeros_like(dts)
eul_CI = np.zeros_like(dts)
mil_means = np.zeros_like(dts)
mil_CI = np.zeros_like(dts)
for i, dt in enumerate(dts):
    MC_est_euler, MC_est_euler_CI, MC_est_mil, MC_est_mil_CI = heston_euler_vs_milstein(M, S0, V0, K, T, dt, r, rho, kappa, theta, xi, seed)
    eul_means[i] = MC_est_euler
    eul_CI[i] = MC_est_euler_CI
    mil_means[i] = MC_est_mil
    mil_CI[i] = MC_est_mil_CI

In [ ]:
plt.figure(figsize=(7,5), dpi=300)
plt.title('Option Price with Euler and Milstein Scheme', fontsize=17)
plt.scatter(dts, mil_means, color='red', label='Milstein Scheme')
plt.fill_between(dts, mil_means - mil_CI, mil_means + mil_CI, alpha=0.4, color='red')
plt.scatter(dts, eul_means, color='green', label='Euler Scheme', marker='v')
plt.fill_between(dts, eul_means - eul_CI, eul_means + eul_CI, alpha=0.5, color='green')
plt.xlabel('dt', fontsize=16)
plt.ylabel('Option Price', fontsize=15)
plt.legend(fontsize=12)
plt.tick_params(axis='both', labelsize=12)
plt.grid(ls='dashed')
plt.xscale('log')
plt.tight_layout()
plt.show()

To verify our implementation, we set $\xi = 0$ and compare it to a geometric Brownian motion benchmark:

In [ ]:
dt = 0.0001
MC_est_euler, MC_est_euler_CI, MC_est_mil, MC_est_mil_CI = heston_euler_vs_milstein(M, S0, V0, K, T, dt, r, rho, kappa, theta, 0, seed)
GBM_mean, GBM_CI = gbm_benchmark(M, S0, K, T, dt, r, sigma, seed)
print(f'Euler (xi=0) price = {MC_est_euler} +- {MC_est_euler_CI}')
print(f'Milstein (xi=0) price = {MC_est_mil} +- {MC_est_mil_CI}')
print(f'GBM price = {GBM_mean} +- {GBM_CI}')

From now on, we use $\Delta t = 10^{-4}$ and the control variate coefficient is set to $c=1$.

In [ ]:
dt = 0.0001
sigma = 0.2
c = 1
M = 100000

The control variate that is used is the analytical geometric Asian price, which we compare to the previous results to ensure accuracy:

In [ ]:
analyt_price = analytical_price(S0, r, T, K, sigma, int(T/dt))
print(f'Control Variate reference (sigma = 0.2, N = 10000, c = 1) = {analyt_price}')
MC_est_euler, MC_est_euler_CI, MC_est_mil, MC_est_mil_CI = heston_euler_vs_milstein(10000, S0, V0, K, T, dt, r, rho, kappa, theta, xi, seed)
print(f'Euler price = {MC_est_euler} +- {MC_est_euler_CI}')
print(f'Milstein price = {MC_est_mil} +- {MC_est_mil_CI}')

Now, we do the first control variate run and compare the prices, standard error, and variance of plain Monte carlo and control variate Monte Carlo

In [ ]:
result_singleM = control_variate_MC(M, S0, V0, K, T, dt, r, rho, kappa, theta, xi, sigma, c, seed)

In [ ]:
print(f"Plain Monte Carlo price = {result_singleM['plain'][0]}. StdErr = {result_singleM['plain'][1]} and Var = {result_singleM['plain'][2]}")
print(f"CV Monte Carlo price = {result_singleM['control_var'][0]}. StdErr = {result_singleM['control_var'][1]} and Var = {result_singleM['control_var'][2]}")

To investigate the efficacy of the variance reduction, we use multiple numbers of paths ($M$):

In [ ]:
for i, M_paths in enumerate([10000, 50000, 100000, 150000, 200000, 250000]):
    result = control_variate_MC(M_paths, S0, V0, K, T, dt, r, rho, kappa, theta, xi, sigma, c, seed*2+i)
    print(f'M = {M_paths} paths')
    print(f"Plain Monte Carlo price = {result['plain'][0]}. StdErr = {result['plain'][1]} and Var = {result['plain'][2]}")
    print(f"CV Monte Carlo price = {result['control_var'][0]}. StdErr = {result['control_var'][1]} and Var = {result['control_var'][2]}")
    print('===============================================================================================================')

We also want to know the impact on the prices and standard errors when varying some Heston parameters. Below we vary $\xi$, $\rho$, and $K$ independently.

In [ ]:
plain_price_xi, plain_std_xi, cv_price_xi, cv_std_xi, xis = heston_sensitivity(M, S0, V0, K, T, dt, r, rho, kappa, theta, xi, sigma, c, seed, parameter_to_vary='xi')

In [ ]:
plain_price_rho, plain_std_rho, cv_price_rho, cv_std_rho, rhos = heston_sensitivity(M, S0, V0, K, T, dt, r, rho, kappa, theta, xi, sigma, c, seed, parameter_to_vary='rho')

In [ ]:
plain_price_k, plain_std_k, cv_price_k, cv_std_k, ks = heston_sensitivity(M, S0, V0, K, T, dt, r, rho, kappa, theta, xi, sigma, c, seed, parameter_to_vary='K')

In [ ]:
def print_results(plain_prices, plain_stds, cv_prices, cv_stds):
    for i in range(len(plain_prices)):
        print(f'Plain MC price = {plain_prices[i]} +- {plain_stds[i]}')
        print(f'Control Variate prices = {cv_prices[i]} +- {cv_stds[i]}')
        print(f'Improvement stderr = {plain_stds[i] / cv_stds[i]}')
        print('===========================')

# print_results(plain_price_xi, plain_std_xi, cv_price_xi, cv_std_xi)
# print_results(plain_price_rho, plain_std_rho, cv_price_rho, cv_std_rho)
print_results(plain_price_k, plain_std_k, cv_price_k, cv_std_k)

In [ ]:
plt.figure(figsize=(18, 6), dpi=300)
plt.suptitle('Parameter Sensitivity Analysis of the Heston Parameters', fontsize=20)
plt.subplot(1, 3, 1)
plt.title(r'Option Price for Different $\xi$', fontsize=17)
plt.errorbar([xi - 0.012 for xi in xis], plain_price_xi, yerr=plain_std_xi, fmt='o', color='blue', label='Plain MC', capsize=6)
plt.errorbar([xi + 0.012 for xi in xis], cv_price_xi, yerr=cv_std_xi, fmt='o', color='red', label='Control Variate MC', capsize=6)
plt.ylabel('Option Price', fontsize=18)
plt.xlabel(r'$\xi$', fontsize=18)
plt.xticks(xis)
plt.tick_params(axis='both', labelsize=14)
plt.legend(fontsize=12)
plt.grid(ls='dashed')

plt.subplot(1, 3, 2)
plt.title(r'Option Price for Different $\rho$', fontsize=17)
plt.errorbar([rho - 0.017 for rho in rhos], plain_price_rho, yerr=plain_std_rho, fmt='o', color='blue', label='Plain MC', capsize=6)
plt.errorbar([rho + 0.017 for rho in rhos], cv_price_rho, yerr=cv_std_rho, fmt='o', color='red', label='Control Variate MC', capsize=6)
plt.xlabel(r'$\rho$', fontsize=18)
plt.xticks(rhos)
plt.tick_params(axis='both', labelsize=14)
plt.legend(fontsize=12)
plt.grid(ls='dashed')

plt.subplot(1, 3, 3)
plt.title(r'Option Price for Different $K$', fontsize=17)
plt.errorbar([k - 0.14 for k in ks], plain_price_k, yerr=plain_std_k, fmt='o', color='blue', label='Plain MC', capsize=6)
plt.errorbar([k + 0.14 for k in ks], cv_price_k, yerr=cv_std_k, fmt='o', color='red', label='Control Variate MC', capsize=6)
plt.xlabel(r'$K$', fontsize=18)
plt.xticks(ks)
plt.tick_params(axis='both', labelsize=14)
plt.legend(fontsize=12)
plt.grid(ls='dashed')

plt.tight_layout()
plt.show()

Lastly, to gain better performance, we estimate the optimal value for the control variate coefficient.

In [ ]:
result = control_variate_MC(M, S0, V0, K, T, dt, r, rho, kappa, theta, xi, sigma, c, seed)

In [ ]:
opt_c = optimal_c(result['payoffs'][1], result['payoffs'][0])
print(f'The optimal values for c is c* = {opt_c}')

In [ ]:
result_singleM_opt_c = control_variate_MC(M, S0, V0, K, T, dt, r, rho, kappa, theta, xi, sigma, opt_c, seed)

In [ ]:
print(f"Plain Monte Carlo price = {result_singleM_opt_c['plain'][0]}. StdErr = {result_singleM_opt_c['plain'][1]} and Var = {result_singleM_opt_c['plain'][2]}")
print(f"CV Monte Carlo price = {result_singleM_opt_c['control_var'][0]}. StdErr = {result_singleM_opt_c['control_var'][1]} and Var = {result_singleM_opt_c['control_var'][2]}")